# Gamess DictionaryFile tutorial

GamessReader provides methods for reading the binary files produced by GAMESS(US) such as the:
* F10
    dictionary file with one electron integrals (details about the contents of this file can be found in the GAMESS documentation

In [1]:
from chemtools.gamessus import DictionaryFile, records, tri2full

`records` is a dictionary of record numbers with descriptions that can be read by the `DictionaryFile` `read_record` method. We can print it's contents 

In [2]:
for k in sorted(records.keys()):
    print k, records[k].name

1 atomic coordinates
2 enrgys
3 gradient vector
4 hessian matrix
5 not used
6 not used
7 ptr
8 dtr
9 ftr
10 gtr
11 bare nucleus
12 overlap
13 kinetic energy
14 alpha fock matrix
15 alpha orbitals
16 alpha density matrix
17 alpha energies or occupation numbers
18 beta fock matrix
19 beta orbitals
20 beta density matrix
21 beta energies or occupation numbers
22 error function interpolation table
23 old alpha fock matrix
24 older alpha fock matrix
25 oldest alpha fock matrix
26 old beta fock matrix
27 older beta fock matrix
28 odest beta fock matrix
29 vib 0 gradient in FORCE
30 vib 0 alpha orbitals in FORCE
31 Vib 0 beta  orbitals in FORCE
32 Vib 0 alpha density matrix in FORCE
33 Vib 0 beta  density matrix in FORCE
34 dipole derivative tensor in FORCE
35 frozen core Fock operator
36 RHF/UHF/ROHF Lagrangian
37 floating point part of common block /OPTGRD/
38 integer part of common block /OPTGRD/
39 ZMAT of input internal coords
40 IZMAT of input internal coords
41 B matrix of redundant in

In [3]:
df = DictionaryFile("data/h2_eq_pvtz_fci.F10")

As a first example we can read the coordianted of the molecule that was used in the calcualtion, in this case H2 with R(H-H)=1.4 bohr.

In [4]:
coords = df.read_record(1)
coords = coords.reshape(2, 3)
print coords

[[ 0.   0.  -0.7]
 [ 0.   0.   0.7]]


Record 2 stores various energies from gamess common block ENRGYS and the contents depend on the type of calcualtion

In [5]:
df.read_record(2)

array([ 0.71428571, -1.88662031, -1.17233459,  0.        ,  0.        ,
        0.        , -1.13296053,  0.        , -2.47528051,  0.5886602 ,
       -3.646691  ,  0.5886602 , -2.34374509,  1.17141049, -1.17233459,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

Record 11 hold the bare nucleus one electron integrals (T + V_ne) as a 1-D vector (lower triangle of a symmetric matrix)
and the low level read_record funtion retrieves that vector

In [6]:
Hv = df.read_record(11)

In [7]:
Hv.shape

(465,)

by using a convenience function tri2full, the 1-D vector can be put in a matrix 

In [10]:
H = tri2full(Hv, (30, 30))

In [11]:
H

array([[ -5.19508256e-01,  -1.00885384e+00,  -6.68935006e-01,
          0.00000000e+00,   0.00000000e+00,  -2.09314765e-01,
          0.00000000e+00,   0.00000000e+00,  -1.30053590e-01,
         -8.31198313e-01,  -8.31198313e-01,  -9.34866370e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -6.34410637e-01,  -7.52595946e-01,  -5.78619781e-01,
          0.00000000e+00,   0.00000000e+00,   4.63303248e-01,
          0.00000000e+00,   0.00000000e+00,   6.58690912e-01,
         -4.64429916e-01,  -4.64429916e-01,  -5.43385761e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -1.00885384e+00,  -1.05786415e+00,  -8.51520320e-01,
          0.00000000e+00,   0.00000000e+00,  -2.22252855e-01,
          0.00000000e+00,   0.00000000e+00,  -2.55072507e-01,
         -6.06115360e-01,  -6.06115360e-01,  -7.78895842e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -7.52595946e-01,  -9.15879535e-01,  -7.79307746e-01,
       

The orbital energies can be read fom record 17

In [19]:
df.read_record(17)

array([-0.59442797,  0.16713829,  0.30507984,  0.64355015,  0.69261917,
        0.69261917,  1.09640445,  1.11998497,  1.11998497,  1.52917244,
        2.33707703,  2.40733557,  3.1753351 ,  3.1753351 ,  3.41225816,
        3.41225816,  3.67481228,  3.89352728,  3.89352728,  4.08443932,
        4.08443932,  4.1141952 ,  4.1141952 ,  4.65705168,  5.72969139,
        5.72969139,  5.8059791 ,  6.96654377,  0.        ,  0.        ])

The molecular orbitals are stored as a vector of stacked columns in record 15

In [24]:
mosv = df.read_record(15);
mosv.shape

(900,)

We can reshape the vector into a matrix by calling the numpy reshape method, remebering to give the order='F' keyword that uses fortran ordering (column by column) instead of default C ordering which is row by row

In [27]:
mos = mosv[:30*28].reshape((30, 28), order='F')

now the first mo can be printed as 

In [26]:
mos[:,0]

array([ 0.18648279,  0.28367175,  0.13546836,  0.        ,  0.        ,
        0.01205087,  0.        ,  0.        ,  0.01741547, -0.00041868,
       -0.00041868,  0.00083736,  0.        ,  0.        ,  0.        ,
        0.18648279,  0.28367175,  0.13546836,  0.        ,  0.        ,
       -0.01205087,  0.        ,  0.        , -0.01741547, -0.00041868,
       -0.00041868,  0.00083736,  0.        ,  0.        ,  0.        ])

In [21]:
mos[:,28]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])